In [22]:
import pandas as pd
df = pd.read_csv('../data/output/csv_outputs/Query4_results.csv')

In [23]:
df.columns

Index(['airline_code', 'departure_airport_code', 'destination_airport_code',
       'departure_date', 'arrival_date', 'ticket_price', 'First_flight',
       'first_flight_code', 'last_flight_code', 'selling_airline',
       'departure_time', 'arrival_time', 'Detected_Language',
       'Assigned_Country', 'Assigned_Currency'],
      dtype='object')

In [19]:
df["ID_Helper"] = df["departure_time"].astype(str) + df["selling_airline"].astype(str) + df["arrival_time"].astype(str)

In [29]:
slim_df = df.drop(['departure_time','selling_airline','arrival_time'], axis = 1)

In [30]:
slim_df

,airline_code,departure_airport_code,destination_airport_code,departure_date,arrival_date,ticket_price,First_flight,first_flight_code,last_flight_code,Detected_Language,Assigned_Country,Assigned_Currency
0,AA,CLT,CUN,"[2024, 6, 26, 7, 15]","[2024, 6, 26, 9, 10]",383,CLT-CUN,AA881,AA881,Deutsch,Deutschland,EUR
1,AA,CLT,CUN,"[2024, 6, 26, 11, 50]","[2024, 6, 26, 13, 54]",383,CLT-CUN,AA886,AA886,Deutsch,Deutschland,EUR
2,AA,CLT,CUN,"[2024, 6, 26, 13, 5]","[2024, 6, 26, 14, 59]",383,CLT-CUN,AA1778,AA1778,Deutsch,Deutschland,EUR
3,AA,CLT,CUN,"[2024, 6, 26, 9, 5]","[2024, 6, 26, 11, 13]",410,CLT-CUN,AA895,AA895,Deutsch,Deutschland,EUR
4,UA,CLT,CUN,"[2024, 6, 26, 10, 7]","[2024, 6, 26, 15, 51]",401,CLT-IAD,UA1361,UA1618,Deutsch,Deutschland,EUR
...,...,...,...,...,...,...,...,...,...,...,...,...
12459,AF,LHR,CDG,"[2024, 11, 13, 11, 30]","[2024, 11, 13, 13, 50]",18825,LHR-CDG,AF1581,AF1581,日本語,日本,JPY
12460,AF,LHR,CDG,"[2024, 11, 13, 14, 45]","[2024, 11, 13, 17, 5]",18825,LHR-CDG,AF1781,AF1781,日本語,日本,JPY
12461,AF,LHR,CDG,"[2024, 11, 13, 17, 35]","[2024, 11, 13, 19, 55]",18825,LHR-CDG,AF1281,AF1281,日本語,日本,JPY
12462,AF,LHR,CDG,"[2024, 11, 13, 19, 45]","[2024, 11, 13, 22, 5]",18825,LHR-CDG,AF1181,AF1181,日本語,日本,JPY
